In [279]:
import pandas as pd
import numpy as np
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font
from openpyxl.styles import Alignment
import time

name_file_data_report = 'Общая таблица слушателей ЦОПП 1-ПО.xlsx' 
path_to_end_folder_report = 'data'

df_po = pd.read_excel(name_file_data_report,sheet_name='ПО',dtype={'Гражданство_получателя_код_страны_по_ОКСМ':str})

#Создаем шрифт которым будем выделять названия таблиц
font_name_table = Font(name='Arial Black',size=15,italic=True)

C:\Users\1\anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [ ]:
pd.

1.3,
2.1.1,
2.1.2,
2.1.3,
2.2 Добавить категории слушателей для 1-ПО
2.3
2.4
2.5
2.6


In [253]:
# Создаем файл excel
wb = openpyxl.Workbook()
# Создаем листы 
wb.create_sheet(title='Раздел 2.4',index=0)
wb.create_sheet(title='Раздел 2.5',index=1)
wb.create_sheet(title='Раздел 2.6',index=2)



<Worksheet "Раздел 2.6">

In [254]:
#Считаем лист 2.4
df_po_2_4 = df_po.copy()
df_po_2_4['for_counting'] = 1

In [255]:
# Считаем общее количество
df_po_2_4_all = pd.pivot_table(df_po_2_4,columns=['Программа_профессионального_обучения_направление_подготовки'],
                              values=['for_counting'],
                              aggfunc='sum')
df_po_2_4_all.index = ['Всего обучено']

In [256]:
df_po_2_4_all

Программа_профессионального_обучения_направление_подготовки,"Программа переподготовки рабочих, служащих","Программа повышения квалификации рабочих, служащих","Программа профессиональной подготовки по профессии рабочего, должности служащего"
Всего обучено,6,1,3


In [257]:
# Считаем по основному общему образованию
df_po_2_4_02 = df_po_2_4[df_po_2_4['Уровень_образования']=='основное общее (9 классов)']


In [258]:
df_po_2_4_02

,Порядковый_номер_группы,Вид_документа,Статус_документа,Подтверждение_утраты,Подтверждение_обмена,Подтверждение_уничтожения,Серия_документа,Номер_документа,Дата_выдачи_документа,Регистрационный_номер,...,Стоимость_обучения_руб,Отметка_об_оплате_оплачено_не_оплачено_в_рамках_индикаторов,Использование_сетевой_формы_обучения,Использование_ЭО_или_ДОТ,прошли_ускоренное_обучение_по_индивидуаль-ным_учебным_планам,Номер_приказа_о_зачислении,Номер_приказа_об_отчислении,Текущий_возраст,Возрастная_категория_1ПО,for_counting
0,1.0,"Свидетельство о профессии рабочего, должности ...",Оригинал,Нет,Нет,Нет,Нет,fa4534,NaN,NaN,...,0.0,в рамках индикаторов,Нет,Без применения ЭО или ДОТ,ускоренное обучение по инд УП,NaN,NaN,17,17 лет,1
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fa4535,NaN,NaN,...,NaN,в рамках индикаторов,Сетевая форма,Исключительно с ЭО ли ДОТ,ускоренное обучение по инд УП,NaN,NaN,18,18 лет,1


In [277]:
# считаем сумму
df_po_2_4_02_all = pd.pivot_table(df_po_2_4_02,columns=['Программа_профессионального_обучения_направление_подготовки'],
                                 values=['for_counting'],
                                 aggfunc='sum')

In [278]:
df_po_2_4_02['Дата_выдачи_оригинала'].dtype

Порядковый_номер_группы        float64
Вид_документа                   object
Статус_документа                object
Подтверждение_утраты            object
Подтверждение_обмена            object
                                ...   
Номер_приказа_о_зачислении     float64
Номер_приказа_об_отчислении    float64
Текущий_возраст                  int64
Возрастная_категория_1ПО        object
for_counting                     int64
Length: 61, dtype: object

In [259]:
#Считаем лист 2.5
# Создаем дополнительную числовую колонку где каждое значение это 1, для удобства агрегирования
df_po_2_5 = df_po.copy()
# Добавляем колонку с 1
df_po_2_5['for_counting'] = 1 


In [260]:
# Считаем строку 01 Всего
df_2_5_all = pd.pivot_table(df_po_2_5,              
              columns=['Программа_профессионального_обучения_направление_подготовки','Источник_финансирования_обучения'],
              values=['for_counting'],
              aggfunc='sum')
df_2_5_all.index = ['Всего']
df_2_5_all.index.name='Код государства по ОКСМ'

In [261]:
df_2_5_all

Программа_профессионального_обучения_направление_подготовки Программа переподготовки рабочих, служащих  \
Источник_финансирования_обучения                                                        Местный бюджет   
Код государства по ОКСМ                                                                                  
Всего                                                                                                1   

Программа_профессионального_обучения_направление_подготовки                   \
Источник_финансирования_обучения                            Платное обучение   
Код государства по ОКСМ                                                        
Всего                                                                      1   

Программа_профессионального_обучения_направление_подготовки                      \
Источник_финансирования_обучения                            Региональный бюджет   
Код государства по ОКСМ                                                           
Всего                                                                         1   

Программа_профессионального_обучения_направление_подготовки                     \
Источник_финансирования_обучения                            Федеральный бюджет   
Код государства по ОКСМ                                                          
Всего                                                                        3   

Программа_профессионального_обучения_направление_подготовки Программа повышения квалификации рабочих, служащих   \
Источник_финансирования_обучения                                                               Платное обучение   
Код государства по ОКСМ                                                                                           
Всего                                                                                                        1    

Программа_профессионального_обучения_направление_подготовки Программа профессиональной подготовки по профессии рабочего, должности служащего   
Источник_финансирования_обучения                                                                                             Платное обучение  
Код государства по ОКСМ                                                                                                                        
Всего                                                                                                        3

In [262]:
# Считаем данные по странам
svod_df_po_2_5=pd.pivot_table(df_po_2_5,
              index=['Гражданство_получателя_код_страны_по_ОКСМ'],
              columns=['Программа_профессионального_обучения_направление_подготовки','Источник_финансирования_обучения'],
              values=['for_counting'],
              aggfunc='sum')

In [263]:
# Удаляем лишний мультииндекс
svod_df_po_2_5.columns= svod_df_po_2_5.columns.droplevel()
# заполняем нулями для корректного суммирования
svod_df_po_2_5.fillna(0.0,inplace=True)


In [264]:
# Соединяем датафреймы
df_po_2_5_out = pd.concat([df_2_5_all,svod_df_po_2_5])
# заменяем нули на нан, чтобы в итоговой таблице нули не отвлекали
df_po_2_5_out.replace(0.0,np.NaN,inplace=True)

In [265]:
df_2_5_out

Программа_профессионального_обучения_направление_подготовки Программа переподготовки рабочих, служащих  \
Источник_финансирования_обучения                                                        Местный бюджет   
Всего                                                                                              1.0   
004                                                                                                NaN   
012                                                                                                NaN   
020                                                                                                NaN   
643                                                                                                1.0   

Программа_профессионального_обучения_направление_подготовки                   \
Источник_финансирования_обучения                            Платное обучение   
Всего                                                                    1.0   
004                                                                      NaN   
012                                                                      NaN   
020                                                                      NaN   
643                                                                      1.0   

Программа_профессионального_обучения_направление_подготовки                      \
Источник_финансирования_обучения                            Региональный бюджет   
Всего                                                                       1.0   
004                                                                         NaN   
012                                                                         NaN   
020                                                                         NaN   
643                                                                         1.0   

Программа_профессионального_обучения_направление_подготовки                     \
Источник_финансирования_обучения                            Федеральный бюджет   
Всего                                                                      3.0   
004                                                                        NaN   
012                                                                        NaN   
020                                                                        NaN   
643                                                                        3.0   

Программа_профессионального_обучения_направление_подготовки Программа профессиональной подготовки по профессии рабочего, должности служащего   
Источник_финансирования_обучения                                                                                             Платное обучение  
Всего                                                                                                      4.0                                 
004                                                                                                        2.0                                 
012                                                                                                        1.0                                 
020                                                                                                        1.0                                 
643                                                                                                        NaN

In [266]:
wb['Раздел 2.5'][f'A1'] = 'Распределение слушателей, обученных по программам ПО, по гражданству'
wb['Раздел 2.5'][f'A1'].font = font_name_table
wb['Раздел 2.5'][f'A2'] = 'В таблице отображаются ТОЛЬКО ТЕ показатели которые присутствуют в ИСХОДНОЙ таблице!!!'
wb['Раздел 2.5'][f'A2'].font = font_name_table



for r in dataframe_to_rows(df_po_2_5_out,index =True,header=True):
    if len(r) != 1:
        wb['Раздел 2.5'].append(r)
wb['Раздел 2.5'].column_dimensions['A'].width = 50
wb['Раздел 2.5'].column_dimensions['B'].width = 50
wb['Раздел 2.5'].column_dimensions['F'].width = 50

In [267]:
# Создаем раздел 2.6
df_po_2_6 = df_po.copy()
df_po_2_6['for_counting'] = 1

In [268]:
# Считаем в общем сколько обучено
df_po_2_6_all = pd.pivot_table(df_po_2_6,columns=['Программа_профессионального_обучения_направление_подготовки','Пол_получателя'],
              values=['for_counting'],
              aggfunc='sum')
df_po_2_6_all.index = ['Всего обучено']

In [269]:
df_po_2_6_all

Программа_профессионального_обучения_направление_подготовки Программа переподготовки рабочих, служащих  \
Пол_получателя                                                                                     Жен   
Всего обучено                                                                                        2   

Программа_профессионального_обучения_направление_подготовки      \
Пол_получателя                                              Муж   
Всего обучено                                                 4   

Программа_профессионального_обучения_направление_подготовки Программа повышения квалификации рабочих, служащих   \
Пол_получателя                                                                                              Жен   
Всего обучено                                                                                                1    

Программа_профессионального_обучения_направление_подготовки Программа профессиональной подготовки по профессии рабочего, должности служащего   
Пол_получателя                                                                                                                            Жен  
Всего обучено                                                                                                3

In [270]:
# Считаем распределение по возрастам
df_po_2_6_by_age = pd.pivot_table(df_po_2_6,index=['Возрастная_категория_1ПО'],
                                 columns=['Программа_профессионального_обучения_направление_подготовки','Пол_получателя'],
                                 values=['for_counting'],
                                 aggfunc='sum')

In [271]:
df_po_2_6_by_age

for_counting  \
Программа_профессионального_обучения_направление_подготовки Программа переподготовки рабочих, служащих   
Пол_получателя                                                                                     Жен   
Возрастная_категория_1ПО                                                                                 
16 лет                                                                                             NaN   
17 лет                                                                                             1.0   
18 лет                                                                                             1.0   
19 лет                                                                                             NaN   
21 год                                                                                             NaN   
45-49 лет                                                                                          NaN   
65 лет и старше                                                                                    NaN   
В возрасте моложе 14 лет                                                                           NaN   

                                                                  \
Программа_профессионального_обучения_направление_подготовки        
Пол_получателя                                               Муж   
Возрастная_категория_1ПО                                           
16 лет                                                       1.0   
17 лет                                                       NaN   
18 лет                                                       NaN   
19 лет                                                       1.0   
21 год                                                       1.0   
45-49 лет                                                    NaN   
65 лет и старше                                              NaN   
В возрасте моложе 14 лет                                     1.0   

                                                                                                                 \
Программа_профессионального_обучения_направление_подготовки Программа повышения квалификации рабочих, служащих    
Пол_получателя                                                                                              Жен   
Возрастная_категория_1ПО                                                                                          
16 лет                                                                                                     NaN    
17 лет                                                                                                     NaN    
18 лет                                                                                                     NaN    
19 лет                                                                                                     NaN    
21 год                                                                                                     NaN    
45-49 лет                                                                                                  NaN    
65 лет и старше                                                                                            NaN    
В возрасте моложе 14 лет                                                                                   1.0    

                                                                                                                                               
Программа_профессионального_обучения_направление_подготовки Программа профессиональной подготовки по профессии рабочего, должности служащего   
Пол_получателя                                                                                                                            Жен  
Возрастная_категория_1ПО                                                                                                                       
16 лет                                     

In [272]:
# Удаляем лишний мультииндекс
df_po_2_6_by_age.columns= df_po_2_6_by_age.columns.droplevel()

In [273]:
# Соединяем 2 датафрейма
df_po_2_6_out = pd.concat([df_po_2_6_all,df_po_2_6_by_age])

In [274]:
df_po_2_6_out

Программа_профессионального_обучения_направление_подготовки Программа переподготовки рабочих, служащих  \
Пол_получателя                                                                                     Жен   
Всего обучено                                                                                      2.0   
16 лет                                                                                             NaN   
17 лет                                                                                             1.0   
18 лет                                                                                             1.0   
19 лет                                                                                             NaN   
21 год                                                                                             NaN   
45-49 лет                                                                                          NaN   
65 лет и старше                                                                                    NaN   
В возрасте моложе 14 лет                                                                           NaN   

Программа_профессионального_обучения_направление_подготовки       \
Пол_получателя                                               Муж   
Всего обучено                                                4.0   
16 лет                                                       1.0   
17 лет                                                       NaN   
18 лет                                                       NaN   
19 лет                                                       1.0   
21 год                                                       1.0   
45-49 лет                                                    NaN   
65 лет и старше                                              NaN   
В возрасте моложе 14 лет                                     1.0   

Программа_профессионального_обучения_направление_подготовки Программа повышения квалификации рабочих, служащих   \
Пол_получателя                                                                                              Жен   
Всего обучено                                                                                              1.0    
16 лет                                                                                                     NaN    
17 лет                                                                                                     NaN    
18 лет                                                                                                     NaN    
19 лет                                                                                                     NaN    
21 год                                                                                                     NaN    
45-49 лет                                                                                                  NaN    
65 лет и старше                                                                                            NaN    
В возрасте моложе 14 лет                                                                                   1.0    

Программа_профессионального_обучения_направление_подготовки Программа профессиональной подготовки по профессии рабочего, должности служащего   
Пол_получателя                                                                                                                            Жен  
Всего обучено                                                                                              3.0                                 
16 лет                                                                                                     NaN                                 
17 лет                                                                                                     NaN                                 
18 лет                                                                                            

In [275]:
wb['Раздел 2.6'][f'A1'] = 'Распределение слушателей, обученных по программам ПО, по возрасту и полу'
wb['Раздел 2.6'][f'A1'].font = font_name_table
wb['Раздел 2.6'][f'A2'] = 'В таблице отображаются ТОЛЬКО ТЕ показатели которые присутствуют в ИСХОДНОЙ таблице!!!'
wb['Раздел 2.6'][f'A2'].font = font_name_table
wb['Раздел 2.6'][f'A3'] = 'Чтобы вычислить показатель ВСЕГО ОБУЧЕНО сложите значения в колонке Муж+Жен'
wb['Раздел 2.6'][f'A3'].font = font_name_table



for r in dataframe_to_rows(df_po_2_6_out,index =True,header=True):
    if len(r) != 1:
        wb['Раздел 2.6'].append(r)
wb['Раздел 2.6'].column_dimensions['A'].width = 50
wb['Раздел 2.6'].column_dimensions['B'].width = 50
wb['Раздел 2.6']['B4'].alignment = Alignment(wrap_text=True)
wb['Раздел 2.6'].column_dimensions['D'].width = 50
wb['Раздел 2.6']['D4'].alignment = Alignment(wrap_text=True)
wb['Раздел 2.6'].column_dimensions['F'].width = 50
wb['Раздел 2.6']['F4'].alignment = Alignment(wrap_text=True)



In [276]:
 # Получаем текущее время для того чтобы использовать в названии
t = time.localtime()
current_time = time.strftime('%H_%M_%S', t)
# Сохраняем итоговый файл
wb.save(f'{path_to_end_folder_report}/Часть отчета 1-ПО.xlsx {current_time}.xlsx')